# Load Graphs

In [5]:
import os
import glob

path="/home/niklas/Documents/Studium_Uni_Bamberg/Semester4/PWM-PT-HS5/data"

def load_graphs(path):
    path_list=[]
    for folder in os.listdir(path):
        os.chdir(path)
        path_list.append(glob.glob(os.path.join(path,folder,'*.{}'.format("edgelist"))))      
    return path_list



## Load Graph for t=1

In [6]:
import networkx as nx

paths=load_graphs(path)
count=len(paths[0])
number=len(paths)

def create_network_t1(paths:list,count:int,number:int):
    edgelist=paths[number-1][count-1]
    G=nx.read_edgelist(edgelist,create_using=nx.DiGraph)
    return G


In [7]:
G=create_network_t1(paths,count,number)
print(G)

DiGraph with 100 nodes and 3044 edges


## Load Graph for t > 1

In [8]:
def create_network_t(paths:list,t:int,run:int):
    G=None
    for i in range(t):
        edgelist=paths[run][i]
        g=nx.read_edgelist(edgelist,create_using=nx.DiGraph)
        if G is None:
            G=g
        else:
            G=nx.compose(G,g)
    return G

In [9]:
G=create_network_t(paths,4,3)
print(G)
in_degrees=dict(G.in_degree(weight="weight"))
in_degrees

DiGraph with 100 nodes and 6266 edges


{'0': 1.4413920914642664,
 '69': 2.78389921208891,
 '55': 0.9128981078675953,
 '7': 0.5851131029154735,
 '6': 2.6219964841904955,
 '57': -5.527009615683059,
 '94': 1.8406603280307021,
 '11': -1.2802191922493498,
 '14': -0.09189598425548984,
 '76': -1.28210181889249,
 '82': 0.7621487119765142,
 '47': 0.11117545183065719,
 '2': 0.805960089980247,
 '42': -2.4845643403243747,
 '90': -0.6213049372171883,
 '45': 0.8784049588775497,
 '71': 1.9332068294328613,
 '87': -0.16669631951023206,
 '91': -2.658761708603193,
 '98': 0.7864972767044245,
 '16': 0.6790964927551263,
 '51': -1.2785208573244131,
 '74': 0.285239997450679,
 '80': -0.39114745213865254,
 '70': 0.8411821536160731,
 '17': -1.2171560992501307,
 '40': -0.9277067295327587,
 '56': -0.18502556879120802,
 '77': 0.053192077120565825,
 '53': -0.9183626512821594,
 '48': 0.3413201022156972,
 '46': -1.9955579685130527,
 '3': 1.8136283946656242,
 '61': 1.4967996972938178,
 '84': -2.7384145142974186,
 '64': 0.5162290767976757,
 '24': 0.677163549

# Load model and agent data

In [10]:
def load_agent_data(path):
    path_list=[]
    for folder in os.listdir(path):
        os.chdir(path)
        path_list.append(glob.glob(os.path.join(path,folder,'agent_data.{}'.format("csv"))))      
    return path_list

agent_paths=load_agent_data(path)

In [11]:
import pandas as pd

def read_csv_t(paths:list,run:int):
    path="%s" % "','".join(paths[run])
    df=pd.read_csv(path)
    return df


In [12]:
df=read_csv_t(agent_paths,4)
df

,Step,AgentID,Info,Avg_PersonalizedTrust,GeneralizedTrust,Wealth,Suspectability,ID,SecurityLevel,Partner,Memory,Threshold
0,2,16,0.000000,-1.701401,-1.770539,0.571352,-0.362987,16,1.021078,50,8,0.702175
1,2,97,0.000000,-0.295401,-0.946604,-2.976599,1.161356,97,-3.490298,90,7,0.789599
2,2,98,0.000000,0.216092,0.840998,-2.088441,0.279939,98,2.416611,69,8,0.673516
3,2,14,0.000000,-0.669487,-2.254114,-3.100177,0.203161,14,0.537178,83,7,-0.059812
4,2,42,0.000000,-0.566904,-0.603277,0.225669,0.146266,42,5.775998,44,4,1.123787
...,...,...,...,...,...,...,...,...,...,...,...,...
9895,100,16,-0.213922,-0.702710,-1.770539,-154.225239,-0.362987,16,1.021078,0,8,0.702175
9896,100,50,0.000000,-0.479525,-1.288465,-160.549927,-0.124293,50,2.072625,32,5,0.533444
9897,100,22,0.000000,-0.162102,-0.708870,-159.672816,0.374204,22,3.689642,6,3,0.292406
9898,100,60,0.593755,-0.335553,-1.131045,-161.182540,0.376700,60,11.798907,71,8,0.706331


In [13]:
def create_degree_df(G:nx.DiGraph,step:int,out:bool):
    if out:
        degree_df = dict(G.out_degree(weight="weight"))
        degree_df = pd.DataFrame(degree_df, index=["out_degree"]).T
        degree_df = degree_df.reset_index()
        degree_df.rename(columns={"index":"ID"},inplace=True)
        degree_df["ID"]=degree_df["ID"].astype(int)
        degree_df["Step"]=step
    else:
        degree_df = dict(G.in_degree(weight="weight"))
        degree_df = pd.DataFrame(degree_df, index=["in_degree"]).T
        degree_df = degree_df.reset_index()
        degree_df.rename(columns={"index":"ID"},inplace=True)
        degree_df["ID"]=degree_df["ID"].astype(int)
        degree_df["Step"]=step
        #degree_df["Step"]=degree_df["Step"].astype(str)

    return degree_df

def merge_df(df1:pd.DataFrame,df2:pd.DataFrame):
    df=pd.merge(df1,df2,on=["Step","ID"])
    return df


In [14]:
def create_pagerank(G:nx.DiGraph, step:int):
    pagerank = nx.pagerank(G)
    pagerank_df = pd.DataFrame(pagerank, index=[0]).T
    pagerank_df.columns = ['pagerank']
    pagerank_df = pagerank_df.reset_index()
    pagerank_df.rename(columns={"index":"ID"},inplace=True)
    pagerank_df["ID"]=pagerank_df["ID"].astype(int)
    pagerank_df["Step"]=step
    return pagerank_df

In [15]:
#loop through data and construct final sets

def analyze_t(path:str):
    for i in range(20):
        agent_paths=load_agent_data(path)
        df=read_csv_t(agent_paths,i)
        paths=load_graphs(path)
        for j in range(100):
            g=create_network_t(paths,j,i)
            if g is not None:
                #calculate out-degrees
                out_degrees = create_degree_df(g,j,out=True)
                in_degrees = create_degree_df(g,j,out=False)
                #pagerank = create_pagerank(g,j)
                df1=merge_df(df,out_degrees)
                df1=merge_df(df1,in_degrees)
                #df1=merge_df(df1,pagerank)
                df1["clustering"]=nx.average_clustering(g)
                df1.to_csv(path+"/"+str(i+1)+"/"+str(j)+"step_data.csv")
                print("finished {}{}".format(i,j))
            
#analyze_t("/home/niklas/Documents/Studium_Uni_Bamberg/Semester4/PWM-PT-HS5/data")
        


# Community detection Girvan-Newman algorithm
- define function for most valuable node
- defined most valuable as node trusted by most agents
- track communities through time
- extract trust network (loop through all nodes and drop all vertices with negative weight)
- run community algorithm

In [16]:
# strip all edges with negative weights out of network
import networkx as nx

def remove_negative_edges(G:nx.DiGraph,path:str,step:int):
    edges_list=[(a,b) for a,b,c in G.edges(data=True) if c["weight"]<=0]
    G.remove_edges_from(edges_list)
    fpath=path+"/"+str(step)+"trust_graph_data.edgelist"
    nx.write_edgelist(G, path=fpath,data=True)


In [18]:
path="/home/niklas/Documents/Studium_Uni_Bamberg/Semester4/PWM-PT-HS5/data"
for i in range(20):
        #current_path=path+"/"+str(i+1)
        gpaths=load_graphs(path)
        for j in range(100):
            G=create_network_t(gpaths,j,i)
            if G is not None:
                  remove_negative_edges(G,path,j)
                  #print("Step:{} done".format(j))
        print("Step:{} done".format(i))
        print("Copied graphs in {}".format(path))


KeyboardInterrupt: 

# Network dfs
- calculate network variables
- create histogram over time

In [25]:
def load_model_data(path:str,step:int):
    path=path+"/"+str(step)+"/"+"model_data.csv"
    data=pd.read_csv(path)
    return data

In [40]:
# Network variables
import numpy as np

path="/home/niklas/Documents/Studium_Uni_Bamberg/Semester4/PWM-PT-HS5/data"

for i in range(20):
    model_data=load_model_data(path,i+1)
    paths=load_graphs(path)
    clustering=[]
    n_nodes=[]
    n_edges=[]
    for j in range(100):
        G=create_network_t(paths,j,i)
        if G is not None:
            clustering.append(nx.average_clustering(G))
            n_nodes.append(G.number_of_nodes())
            n_edges.append(G.size())
            print("finished step:",j)
    model_data["avgClustering"]=pd.Series(clustering)
    model_data["edges"]=pd.Series(n_edges)
    model_data["nodes"]=pd.Series(n_nodes)



KeyboardInterrupt: 